In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import _tree


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
df = pd.read_csv("/Users/carolbarroco/Documents/RecomendSisKNN/TG2 RecommendSys/GitHub/T2-Indroducao-Ciencia-de-Dados/Data/MPBROCKMETAL_KGDataSet.csv")

In [ ]:

# --- 1. Prepare your data ---
# X_scaled = your features (already scaled if needed)
# y = your target labels (e.g., genre)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# --- 2. Train the Random Forest ---
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
model.fit(X_train, y_train)

# --- 3. Find the best single tree ---
best_score = 0
best_tree_index = 0

for i, tree in enumerate(model.estimators_):
    y_pred = tree.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    if acc > best_score:
        best_score = acc
        best_tree_index = i

print(f"Best tree index: {best_tree_index}")
print(f"Best tree accuracy: {best_score:.4f}")

best_tree = model.estimators_[best_tree_index]

# --- 4. Extract decision rules from the best tree ---
def extract_rules(tree, feature_names, X_data, index_data):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    rules = []
    
    def recurse(node, depth, data_indices):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            gini = tree_.impurity[node]
            left = tree_.children_left[node]
            right = tree_.children_right[node]
            rules.append({
                "feature": name,
                "threshold": threshold,
                "gini": gini,
                "depth": depth,
                "node": node,
                "data_indices": data_indices
            })
            left_indices = data_indices[X_data.loc[data_indices, name] <= threshold]
            right_indices = data_indices[X_data.loc[data_indices, name] > threshold]
            recurse(left, depth + 1, left_indices)
            recurse(right, depth + 1, right_indices)
    
    recurse(0, 0, index_data)
    return rules

# Call it with best tree
rules = extract_rules(best_tree, feature_names=X_scaled.columns, X_data=X_scaled, index_data=X_scaled.index)
rules_df = pd.DataFrame(rules)

print("\nSample of extracted rules:")
print(rules_df.head())

# --- 5. Compare full forest vs. best tree ---
forest_acc = accuracy_score(y_val, model.predict(X_val))
tree_acc = accuracy_score(y_val, best_tree.predict(X_val))

print(f"\nRandom Forest accuracy: {forest_acc:.4f}")
print(f"Best Tree accuracy:     {tree_acc:.4f}")
